In [1]:
import pandas as pd
import numpy as np
import textrank
import sklearn
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
col_labels = ['Date','AppID','AppName','Lang','1', '2', '3', 'Author','Stars','Title','Review','Reply','Device','DeviceType','Tags','Updated']

In [3]:
df = pd.read_csv('reviews_googleplay_1507788606.csv', 
                  skiprows=4, engine='python', names = col_labels, index_col=False,
                  error_bad_lines=True
                 )

In [4]:
df_1 = df.drop(['AppID','AppName', '1', '2', '3', 'Device','DeviceType','Tags','Updated'], axis =1)

In [5]:
df_1.head()

,Date,Lang,Author,Stars,Title,Review,Reply
0,\t2017-10-11,pt,Renato Morais,5,NaN,Muito bom,NaN
1,\t2017-10-11,de,NaN,4,NaN,Scheint eine gute App zu sein. Sehr übersichtl...,NaN
2,\t2017-10-11,pt,NaN,5,NaN,Perfeito. Consigo controlar todos os dados imp...,NaN
3,\t2017-10-11,ru,Eva Lollipop,5,NaN,"Огромное спасибо, за ваш труд! Это лучше прило...",NaN
4,\t2017-10-11,es,NaN,5,NaN,Excelente,NaN


In [6]:
from yandex_translate import YandexTranslate
translate = YandexTranslate('trnsl.1.1.20171012T153315Z.27f7b7ad41361c04.5bb573e2c4533a4aeafa7deeba56e203de72b930')
print('Languages:', translate.langs)
print('Translate directions:', translate.directions)
print('Detect language:', translate.detect('Привет, мир!'))
print('Translate:', translate.translate('Привет, мир!', 'ru-en'))  # or just 'en'

Languages: {'fi', 'be', 'cs', 'pt', 'tr', 'sv', 'az', 'hy', 'sk', 'es', 'nl', 'lv', 'no', 'uk', 'en', 'el', 'de', 'fr', 'ca', 'ro', 'it', 'ru', 'mk', 'lt', 'pl', 'sl', 'sr', 'hr', 'et', 'hu', 'sq', 'da', 'bg'}
Translate directions: ['az-ru', 'be-bg', 'be-cs', 'be-de', 'be-en', 'be-es', 'be-fr', 'be-it', 'be-pl', 'be-ro', 'be-ru', 'be-sr', 'be-tr', 'bg-be', 'bg-ru', 'bg-uk', 'ca-en', 'ca-ru', 'cs-be', 'cs-en', 'cs-ru', 'cs-uk', 'da-en', 'da-ru', 'de-be', 'de-en', 'de-es', 'de-fr', 'de-it', 'de-ru', 'de-tr', 'de-uk', 'el-en', 'el-ru', 'en-be', 'en-ca', 'en-cs', 'en-da', 'en-de', 'en-el', 'en-es', 'en-et', 'en-fi', 'en-fr', 'en-hu', 'en-it', 'en-lt', 'en-lv', 'en-mk', 'en-nl', 'en-no', 'en-pt', 'en-ru', 'en-sk', 'en-sl', 'en-sq', 'en-sv', 'en-tr', 'en-uk', 'es-be', 'es-de', 'es-en', 'es-ru', 'es-uk', 'et-en', 'et-ru', 'fi-en', 'fi-ru', 'fr-be', 'fr-de', 'fr-en', 'fr-ru', 'fr-uk', 'hr-ru', 'hu-en', 'hu-ru', 'hy-ru', 'it-be', 'it-de', 'it-en', 'it-ru', 'it-uk', 'lt-en', 'lt-ru', 'lv-en', 'l

## Перевод русских отзывов 

In [10]:
russian_text_df = df_1[df_1['Lang'] == 'ru']  
ru_length = len(russian_text_df) # количество русских отзывов 
p = 51 # проверенная порция для яндекс перевода 
cycles = ru_length // p # кол-во полных циклов 
cyc_tail = ru_length - (cycles*p) #остаток хвоста 

russian_list =[] #список, где складываем все тексты 

x = 0 #начало интервала 
cycles_count = 0 # количество порций / итераций 


for i in range (cycles):
    # выбираем русский текст и запускаем первый цикл начинаем от 0 до 51 
    text_portion = russian_text_df['Review'][x:p] 
    # переводим его через яндекс и выбираем из него словарь текстов 
    russian = translate.translate(text_portion, 'ru-en')['text']
    #добавляем его в общий словарь 
    russian_list.append(russian)
    # готов координаты для следующей порции 
    x = p
    p = p + 50
    cycles_count += 1
    
    # добавляем хвост, который не вошел в цикл (до 50)
    
    if cycles_count == cycles:
        x = p
        text_portion_1 = russian_text_df['Review'][x:]
        russian = translate.translate(text_portion_1, 'ru-en')['text']
        russian_list.append(russian)

print (len(russian_list))

YandexTranslateException: None